In [2]:
import numpy as np
import random

Li, Bramley, & Gureckis (2021)
===

Trying to recreate the suspense modelling from Li, Bramley, and Gureckis (2021).

I'll copy over some of their code from ([OSF](https://osf.io/zsjk8)), and rewrite some for my own clarity

The minimal amount to get their code working and simulating suspense is:
- A belief model: the player's perceived probability of success
- A suspense model: the metric that quantifies suspense as a function of belief

In [3]:
# Generate a sequence of pairs

def genPairSequence(theGame: dict) -> dict:
    # Deconstruct into upper and lower bounds
    upperBound, lowerBound = theGame['bounds']

    # Define callbacks that we'll use to check if the player is bust or not
    if theGame['isBust']:
        # Check if score is above upperBound or below lowerBound
        gameDone = lambda x: x >= upperBound or x <= lowerBound
    else:
        gameDone = lambda x: False

    pairSequence = []
    for n in range(theGame['nmax']):
        # Sample from the deck twice, without replacement - i.e. draw 2 unique cards (although their value CAN be equal)
        pairSequence.append(np.random.choice(theGame['deck'], 2, replace=False))

        # Check if the player is bust after drawing a new pair - note that they'll always get the first card in the pair as it's fixed
        if gameDone(sum(p[0] for p in pairSequence)):
            print('Bust')
            break

    theGame['pairSequence'] = np.array(pairSequence)
    theGame['isWin'] = checkIfWon([pr[0] for pr in pairSequence], theGame)

    return theGame

def checkIfWon(cards: list, theGame: dict) -> int:
    # Deconstruct bounds
    upperBound, lowerBound = theGame['bounds']
    # Check if the player is bust - NB: this counts bust as inclusive of the upperBound, unlike in actual Blackjack
    bust = (sum(cards) >= upperBound) or (sum(cards) <= lowerBound)
    # Define a default value of outcome for debugging
    outcome = 0

    if bust:
        # if the player is bust, or if they've exceeded the max number of draws
        # I have a feeling this check is actually redundant, but I'll leave it in for consistency with the authors
        if theGame['isBust'] or (not theGame['isBust'] and len(cards) == theGame['nmax']):
            # Lost
            outcome = -1
    else:
        # If all cards have been drawn without going bust
        if len(cards) == theGame['nmax']:
            # Win
            outcome = 1

    return outcome


In [ ]:
# Suspense function

def computeSuspense(cPair, cPast, mod, theGame):
    pass

In [4]:
# Main
# ----------------------------------------------------
# ## Define game parameters
nmax = 5 # Maximum number of cards that can be drawn
deckSize = 9 # The size of the deck
allowedCards = np.arange(-3, 10) # The possible cards - NB: including negative values! - from which each real deck is sampled
# -----------------------------------------------------

# Define game parameter storage object
theGame = {
    'bounds' : [21, -100], # the upper and lower bounds of the game score - eg: over 21, the player goes bust,
    'nmax' : nmax,
    'deck' : np.random.choice(allowedCards, deckSize), # samples the allowed cards, `deckSize` times, with replacement
    'isBust' : True
    }

# Generate the sequence of card pairs
theGame = genPairSequence(theGame)

# Compute the suspense for this game
computeSuspense(theGame,['L1','tobnd','plose','uncertainty'])

In [5]:
theGame

{'bounds': [21, -100],
 'nmax': 5,
 'deck': array([ 4,  5,  1,  9,  7, -2,  0,  9,  4]),
 'isBust': True,
 'pairSequence': array([[ 1,  4],
        [-2,  7],
        [ 7,  4],
        [ 4,  5],
        [ 1,  7]]),
 'isWin': 1}